### Normalize Kaggle Bike Sharing Dataset

Normalize 'temp','atemp','humidity','windspeed' and store the train and test files

In [3]:
!pip install matplotlib

     |████████████████████████████████| 9.2 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 13.0 MB/s eta 0:00:01
     |████████████████████████████████| 103 kB 15.4 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 10.6 MB/s eta 0:00:01


In [2]:
!pip install scikit-learn

     |████████████████████████████████| 11.1 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 34.5 MB 12.3 MB/s eta 0:00:01


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA

import boto3
import sagemaker.amazon.common as smac

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/saravanan/.config/sagemaker/config.yaml


In [4]:
columns = ['count', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'year', 'month', 'day', 'dayofweek','hour']

cols_normalize = ['temp','atemp','humidity','windspeed']

In [5]:
df = pd.read_csv('train.csv', parse_dates=['datetime'])
df_test = pd.read_csv('test.csv', parse_dates=['datetime'])

In [6]:
# convert datetime into multiple feautures

def add_features(df):
    df['year']=df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['dayofweek']=df['datetime'].dt.dayofweek
    df['hour']=df['datetime'].dt.hour

In [7]:
add_features(df)
add_features(df_test)

In [8]:
df['count'] = df['count'].map(np.log1p)

In [9]:
df.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,dayofweek,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,2.833213,2011,1,1,5,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,3.713572,2011,1,1,5,1


In [10]:
df_test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,3,0
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,1
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,2
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,3
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,4


In [11]:
# initiating the normalizer

transformer = Normalizer()

In [12]:
transformer.fit(df[cols_normalize])

Normalizer()

In [14]:
transformer.transform(df[cols_normalize])

array([[0.11876091, 0.17373611, 0.97760504, 0.        ],
       [0.11046696, 0.16698636, 0.9797513 , 0.        ],
       [0.11046696, 0.16698636, 0.9797513 , 0.        ],
       ...,
       [0.21030903, 0.24002989, 0.92029059, 0.22632058],
       [0.21370435, 0.26713044, 0.93514817, 0.09203084],
       [0.18768111, 0.2383922 , 0.94412752, 0.12871748]])

In [15]:
def transform_data(scaler, df, columns):
    transformed_data = scaler.transform(df[columns])
    df_transformed = pd.DataFrame(transformed_data, columns=columns)

    for col in df_transformed.columns:
        df[col]=df_transformed[col]

In [16]:
transform_data(transformer, df, cols_normalize)
transform_data(transformer, df_test, cols_normalize)

In [17]:
df.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,dayofweek,hour
0,2011-01-01 00:00:00,1,0,0,1,0.118761,0.173736,0.977605,0.0,3,13,2.833213,2011,1,1,5,0
1,2011-01-01 01:00:00,1,0,0,1,0.110467,0.166986,0.979751,0.0,8,32,3.713572,2011,1,1,5,1


In [18]:
df_test.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour
0,2011-01-20 00:00:00,1,0,1,1,0.167404,0.178475,0.879419,0.408344,2011,1,20,3,0
1,2011-01-20 01:00:00,1,0,1,1,0.181869,0.232625,0.955410,0.000000,2011,1,20,3,1


In [19]:
# save normalized train and test data
df.to_csv('train_normalized.csv',index=False, columns=columns)
df_test.to_csv('test_normalized.csv',index=False)

In [20]:
# Save only the 4 normalized numeric columns for PCA Training and Test

In [26]:
def write_recordio_file(filename, x, y=None):
    with open(filename,'wb') as f:
        smac.write_numpy_to_dense_tensor(f,x,y)

In [28]:
# SAve alll normalized data as RecordIO file for PCA training in sagemaker

X = df[['temp','atemp','humidity','windspeed']].values
write_recordio_file('bike_train_numeric_columns.recordio',X)